In [55]:
import pandas as pd
import numpy as np
import os

In [56]:
list_of_df = []
n_rnds = 5

for rnd in range(n_rnds):
    epoch_string = 'bert-large/20_epochs_baseline/saved_test_result_1b/bert-large-uncased/'
    epoch_string += str(rnd) + '/'
    epoch_location = epoch_string + 'unformatted_test_result.tsv'
    print(epoch_location)
    list_of_df.append(pd.read_csv(epoch_location, sep='\t'))


for rnd in range(n_rnds):
    epoch_string = 'bert-large/20_epochs_baseline/saved_test_result_1b_2/bert-large-uncased/'
    epoch_string += str(rnd) + '/'
    epoch_location = epoch_string + 'unformatted_test_result.tsv'
    print(epoch_location)
    list_of_df.append(pd.read_csv(epoch_location, sep='\t'))

for rnd in range(n_rnds):
    epoch_string = 'bert-large/20_epochs_baseline/saved_test_result_1b_3/bert-large-uncased/'
    epoch_string += str(rnd) + '/'
    epoch_location = epoch_string + 'unformatted_test_result.tsv'
    print(epoch_location)
    list_of_df.append(pd.read_csv(epoch_location, sep='\t'))

for rnd in range(n_rnds):
    epoch_string = 'bert-large/20_epochs_baseline/saved_test_result_1b_4/bert-large-uncased/'
    epoch_string += str(rnd) + '/'
    epoch_location = epoch_string + 'unformatted_test_result.tsv'
    print(epoch_location)
    list_of_df.append(pd.read_csv(epoch_location, sep='\t'))

for rnd in range(n_rnds):
    epoch_string = 'bert-large/20_epochs_baseline/saved_test_result_1b_4/bert-large-uncased/'
    epoch_string += str(rnd) + '/'
    epoch_location = epoch_string + 'unformatted_test_result.tsv'
    print(epoch_location)
    list_of_df.append(pd.read_csv(epoch_location, sep='\t'))


bert-large/20_epochs_baseline/saved_test_result_1b/bert-large-uncased/0/unformatted_test_result.tsv
bert-large/20_epochs_baseline/saved_test_result_1b/bert-large-uncased/1/unformatted_test_result.tsv
bert-large/20_epochs_baseline/saved_test_result_1b/bert-large-uncased/2/unformatted_test_result.tsv
bert-large/20_epochs_baseline/saved_test_result_1b/bert-large-uncased/3/unformatted_test_result.tsv
bert-large/20_epochs_baseline/saved_test_result_1b/bert-large-uncased/4/unformatted_test_result.tsv
bert-large/20_epochs_baseline/saved_test_result_1b_2/bert-large-uncased/0/unformatted_test_result.tsv
bert-large/20_epochs_baseline/saved_test_result_1b_2/bert-large-uncased/1/unformatted_test_result.tsv
bert-large/20_epochs_baseline/saved_test_result_1b_2/bert-large-uncased/2/unformatted_test_result.tsv
bert-large/20_epochs_baseline/saved_test_result_1b_2/bert-large-uncased/3/unformatted_test_result.tsv
bert-large/20_epochs_baseline/saved_test_result_1b_2/bert-large-uncased/4/unformatted_test_r

In [57]:
len(list_of_df)

25

In [58]:
original_df = pd.read_csv('Datasets/Subtask_1b/test.tsv', sep='\t')
original_df

,tweet_id,tweet,class
0,id,text,class
1,SMM4H2022UsOrVPnHjHXTdciW,i think my xarelto is causing bruising...it mi...,ADE
2,SMM4H2022nqSx5YKgncBmaJ40,“@USER_________: Adderall had me staring at my...,ADE
3,SMM4H20227fgc68K2dIAbvPLj,@USER________ advair really messes you up jesu...,ADE
4,SMM4H20226SwdPJeIwdtgHNlJ,How tf does my body handle vyvanse and adderal...,ADE
...,...,...,...
635,SMM4H2022KLJbFODskPbJXjNN,@USER________ saphris is the one that finally ...,ADE
636,SMM4H2022EP1zjMNzSpcaaibK,@USER___________ mine are okay. i just feel a ...,ADE
637,SMM4H20220oUnNG4V4ADSYxzH,i got miralax and now i have a serve headache,ADE
638,SMM4H2022YQqnJH5iIdwJr0Dz,@USER__________ yep correct. took it for 2 yea...,ADE


In [59]:
best_results = []

for index, row in original_df.iterrows():
    # getting the original values in the tweet (to minimize shuffling)
    original_tweet_id = row['tweet_id']
    
    
    # go through every epoch's row of data 
    specific_row_result = []
    for epoch in range(len(list_of_df)): 
        row = list_of_df[epoch].loc[(list_of_df[epoch]['id'] == original_tweet_id)]
        # print(row)
        # retrieving the length of the span
        predicted_begin = row['predict_begin'].values[0]
        predicted_end = row['predict_end'].values[0]
        span_length = predicted_end - predicted_begin
        
        if span_length == 0:
            continue

        specific_row_result.append((epoch, original_tweet_id, span_length))
    
    # sort the specific_row_results by increasing span length
    specific_row_result.sort(key = lambda x : x[2])
    best_results.append(specific_row_result)

# go through every specific_row_results and keeping the median value ONLY
for row in range(len(best_results)):
    
    length_specific_result = len(best_results[row])
    
    if length_specific_result == 0: 
        continue

    median_of_length_specific_result = int(length_specific_result / 2)

    best_results[row] = best_results[row][median_of_length_specific_result]

# appending all the results to create one final result
unformatted_merged_result = pd.DataFrame(columns = ['id', 'text', 'class', 'predict_begin', 'predict_end', 'predict_span'])

for row in range(len(best_results)):
    if best_results[row] == []:
        # retrieving it from the original dataframe
        original_row_result = original_df.iloc[row]
        tweet_id = original_row_result['tweet_id']
        orig_sentence = original_row_result['tweet']
        orig_class = original_row_result['class']
        len_orig_sentence = len(orig_sentence)
        
        # creating a dataframe row and adding the lengths of the original sentences to the result
        
        to_append = pd.DataFrame({'id': [tweet_id], 'text': [orig_sentence], 'class': [orig_class], 'predict_span': [orig_sentence], 'predict_begin': [0], 'predict_end': [len_orig_sentence]})
       
        unformatted_merged_result = unformatted_merged_result.append(to_append, ignore_index=True)

        continue
    
    particular_df = list_of_df[best_results[row][0]]
    original_row_result = particular_df.loc[(particular_df['id'] == best_results[row][1])]
  
    tweet_id = original_row_result['id'].values[0]
    orig_sentence = original_row_result['text'].values[0]
    orig_class = 'ADE'
    predict_span = original_row_result['predict_span'].values[0]
    predict_begin = original_row_result['predict_begin'].values[0]
    predict_end = original_row_result['predict_end'].values[0]

    to_append = pd.DataFrame({'id': [tweet_id], 'text': [orig_sentence], 'class': [orig_class], 'predict_begin': [predict_begin], 'predict_end': [predict_end], 'predict_span': [predict_span]})
    unformatted_merged_result = unformatted_merged_result.append(to_append, ignore_index=True)



In [60]:
unformatted_merged_result = unformatted_merged_result.iloc[1:]
unformatted_merged_result

,id,text,class,predict_begin,predict_end,predict_span
1,SMM4H2022UsOrVPnHjHXTdciW,i think my xarelto is causing bruising...it mi...,ADE,30,38,bruising
2,SMM4H2022nqSx5YKgncBmaJ40,“@USER_________: Adderall had me staring at my...,ADE,22,39,staring at myself
3,SMM4H20227fgc68K2dIAbvPLj,@USER________ advair really messes you up jesu...,ADE,68,93,ripping their clothes off
4,SMM4H20226SwdPJeIwdtgHNlJ,How tf does my body handle vyvanse and adderal...,ADE,96,123,feel like i am going to die
5,SMM4H2022grb3peMyLn0aZM4G,really should start taking my medication again...,ADE,78,96,straighten my hair
...,...,...,...,...,...,...
635,SMM4H2022KLJbFODskPbJXjNN,@USER________ saphris is the one that finally ...,ADE,105,117,not sleeping
636,SMM4H2022EP1zjMNzSpcaaibK,@USER___________ mine are okay. i just feel a ...,ADE,41,76,drunk sometimes i use to get spinny
637,SMM4H20220oUnNG4V4ADSYxzH,i got miralax and now i have a serve headache,ADE,37,45,headache
638,SMM4H2022YQqnJH5iIdwJr0Dz,@USER__________ yep correct. took it for 2 yea...,ADE,41,45,hell


In [61]:
# Saving the result
formatted_merged_result = unformatted_merged_result.drop(columns = ['text'])

os.makedirs('bert-large/20_epochs_baseline/final_test_result/', exist_ok=True)
unformatted_merged_result.to_csv('bert-large/20_epochs_baseline/final_test_result/unformatted_merged_result.tsv', sep='\t', index=False)
formatted_merged_result.to_csv('bert-large/20_epochs_baseline/final_test_result/formatted_merged_result.tsv', sep='\t', index=False, header=False)

